In [1]:
import sys 

sys.path.insert(0, '../src/')

In [2]:
from data.data_load import DataLoad

In [3]:
dl = DataLoad()

In [4]:
df = dl.load_data('train_dataset_name')

2023-11-27 20:14:17 [info     ] Comecando a carga dos dados com o nome: train_dataset_name


In [5]:
df.head()

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [6]:
df.shape

(150000, 11)

In [7]:
from data.data_validation import DataValidation

In [8]:
dv = DataValidation()

In [9]:
is_valid = dv.run(df)

2023-11-27 20:14:25 [info     ] Validacao iniciou
2023-11-27 20:14:25 [info     ] Validation columns passed...
2023-11-27 20:14:25 [info     ] Validacao com sucesso.


In [10]:
from data.data_transformation import DataTransformation

In [11]:
dt = DataTransformation(df)

In [12]:
X_train, X_valid, y_train, y_valid = dt.train_test_spliting()

In [13]:
X_train.head()

,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
146433,0.183822,30,0,0.176638,5983.0,5,0,2,0,1.0
15597,0.533493,40,0,0.146019,2800.0,2,0,0,0,0.0
111605,1.026997,34,0,0.065518,4700.0,1,0,0,0,0.0
85418,0.017300,83,0,19.000000,NaN,4,0,0,0,0.0
9652,0.682460,61,0,0.140232,5333.0,3,0,0,0,2.0


In [15]:
from sklearn.pipeline import Pipeline
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler

from utils.utils import load_config_file

In [16]:
pipe = Pipeline([('imputer', MeanMedianImputer(variables=load_config_file().get('vars_imputer'))),
                 ('discretizer', EqualFrequencyDiscretiser(variables=load_config_file().get('vars_discretizer'))),
                 ('scaler', SklearnTransformerWrapper(StandardScaler()))])

In [18]:
from data.data_preprocess import DataPreprocess

In [19]:
preprocessor = DataPreprocess(pipe)

In [20]:
preprocessor.train(X_train)

2023-11-27 20:29:20 [info     ] Preprocessamento iniciou...


In [21]:
X_train_processed = preprocessor.transform(X_train)

2023-11-27 20:29:35 [info     ] Transformacao dos dados com preprocessador iniciou...


In [22]:
X_valid_processed = preprocessor.transform(X_valid)

2023-11-27 20:29:48 [info     ] Transformacao dos dados com preprocessador iniciou...


In [23]:
X_train_processed.shape

(100500, 10)

In [24]:
X_valid_processed.shape

(49500, 10)

In [25]:
from train.train import TrainModels

In [26]:
tm = TrainModels(dados_X=X_train_processed,
                 dados_y=y_train)

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model = tm.train(model=LogisticRegression())

In [29]:
from evaluation.classifier_eval import ModelEvaluation

In [30]:
model_eval = ModelEvaluation(model,
                             X_train_processed,
                             y_train,
                             n_splits=5)
roc_auc_scores = model_eval.cross_val_evaluate()

2023-11-27 20:46:22 [info     ] Iniciou a validacao cruzada...


In [32]:
roc_auc_scores.mean()

0.8055819308011325